### HYPERPARAMETER OPTIMIZATION
 1. GridSearch
    xtreristics:
        - searches through the entire parameter distribution.
        - can be slow and very computationally expensive.
        - It is the most powerful in terms of finding the optimal hyper-params
 2. Randomized Search
    xteristics:
        - searches through a randomly sampled subset of the hyper-params where sample is size = number of iterations.
        - can be significantly faster than grid search based on the value of n_iter.
        - It is the poorest in terms of finding the optimal hyper-params
 3. Bayesian Optimization.
    xteristics:
        - uses conditional probability (bayes theorem) to conduct the search of the optimal params
        - it is significantly faster and less computationally expensive than gridsearch
        - it is more powerful than randomized search in terms of finding the optimal hyper params
    - Optuna
    - HyperOpt

### GRID SERACH

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
X = load_diabetes(as_frame=True)['data']
y = load_diabetes()['target']

In [12]:
# init the estimator 
rfr_estimator = RandomForestRegressor(random_state=23)

# init the fold
k_fold = KFold(n_splits=5, shuffle=True, random_state=23)

params= {
    'n_estimators' : [x for x in range(5, 101) if x % 10 == 0],
    'max_depth' : [x for x in range(2, 10)],
    'criterion': ['squared_error', 'poisson','absolute_error']}

model = GridSearchCV(estimator=rfr_estimator, param_grid=params,
                     scoring = 'neg_root_mean_squared_error', n_jobs= 1,
                     verbose= 3, return_train_score= True, cv= k_fold.split(X))
model.fit(X = X, y = y)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
[CV 1/5] END criterion=squared_error, max_depth=2, n_estimators=10;, score=(train=-53.770, test=-60.053) total time=   0.0s
[CV 2/5] END criterion=squared_error, max_depth=2, n_estimators=10;, score=(train=-55.515, test=-53.994) total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=2, n_estimators=10;, score=(train=-53.936, test=-59.190) total time=   0.0s
[CV 4/5] END criterion=squared_error, max_depth=2, n_estimators=10;, score=(train=-54.312, test=-57.260) total time=   0.1s
[CV 5/5] END criterion=squared_error, max_depth=2, n_estimators=10;, score=(train=-54.204, test=-58.787) total time=   0.0s
[CV 1/5] END criterion=squared_error, max_depth=2, n_estimators=20;, score=(train=-53.892, test=-59.471) total time=   0.3s
[CV 2/5] END criterion=squared_error, max_depth=2, n_estimators=20;, score=(train=-55.192, test=-53.670) total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=2, n_estimators=20;,

GridSearchCV(cv=<generator object _BaseKFold.split at 0x0000025CF11B88C0>,
             estimator=RandomForestRegressor(random_state=23), n_jobs=1,
             param_grid={'criterion': ['squared_error', 'poisson',
                                       'absolute_error'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             return_train_score=True, scoring='neg_root_mean_squared_error',
             verbose=3)

In [13]:
model.best_estimator_

RandomForestRegressor(max_depth=3, n_estimators=40, random_state=23)

In [14]:
model.best_params_

{'criterion': 'squared_error', 'max_depth': 3, 'n_estimators': 40}

In [16]:
-model.best_score_

56.89101506653416

### RANDOMIZED SEARCH

In [17]:
# init the random_search object

model = RandomizedSearchCV(estimator=rfr_estimator, param_distributions= params,
                           n_iter= 50, scoring= 'neg_root_mean_squared_error',
                           verbose = 3, random_state= 23, return_train_score=True,
                           cv= k_fold.split(X), n_jobs = 1)
model.fit(X, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END criterion=poisson, max_depth=2, n_estimators=30;, score=(train=-54.335, test=-59.356) total time=   0.1s
[CV 2/5] END criterion=poisson, max_depth=2, n_estimators=30;, score=(train=-55.710, test=-55.994) total time=   0.1s
[CV 3/5] END criterion=poisson, max_depth=2, n_estimators=30;, score=(train=-53.793, test=-60.805) total time=   0.0s
[CV 4/5] END criterion=poisson, max_depth=2, n_estimators=30;, score=(train=-54.627, test=-57.677) total time=   0.1s
[CV 5/5] END criterion=poisson, max_depth=2, n_estimators=30;, score=(train=-54.758, test=-60.187) total time=   0.1s
[CV 1/5] END criterion=squared_error, max_depth=2, n_estimators=60;, score=(train=-53.420, test=-59.322) total time=   0.1s
[CV 2/5] END criterion=squared_error, max_depth=2, n_estimators=60;, score=(train=-55.045, test=-53.673) total time=   0.0s
[CV 3/5] END criterion=squared_error, max_depth=2, n_estimators=60;, score=(train=-53.468, test=-59.

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000025CF11B8590>,
                   estimator=RandomForestRegressor(random_state=23), n_iter=50,
                   n_jobs=1,
                   param_distributions={'criterion': ['squared_error',
                                                      'poisson',
                                                      'absolute_error'],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   random_state=23, return_train_score=True,
                   scoring='neg_root_mean_squared_error', verbose=3)

In [5]:
[x for x in range(2, 10)]

[2, 3, 4, 5, 6, 7, 8, 9]

### BAYESIAN OPTIMIZATION WITH OPTUNA

In [19]:
import optuna
from sklearn.model_selection import cross_val_score


# define the objective function

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estiimators', 10, 101)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    criterion = trial.suggest_categorical('criterion', ['absolute_error','poisson','squared_error'])
    
    model = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth,
                                  criterion=criterion, random_state=23)
    score = -cross_val_score(estimator=model, X=X, y = y, scoring='neg_root_mean_squared_error',
                             cv= k_fold.split(X)).mean()
    
    return score

# init the study
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(seed=23))
# optimize the study
study.optimize(objective, n_trials=50)

[I 2025-04-04 14:33:40,497] A new study created in memory with name: no-name-5fb73aca-3c51-4c54-b3e1-48d4fb50cfed
[I 2025-04-04 14:33:44,943] Trial 0 finished with value: 58.14466023072968 and parameters: {'n_estiimators': 57, 'max_depth': 10, 'criterion': 'absolute_error'}. Best is trial 0 with value: 58.14466023072968.
[I 2025-04-04 14:33:46,016] Trial 1 finished with value: 57.91427367693552 and parameters: {'n_estiimators': 73, 'max_depth': 3, 'criterion': 'poisson'}. Best is trial 1 with value: 57.91427367693552.
[I 2025-04-04 14:33:46,678] Trial 2 finished with value: 58.53303179991748 and parameters: {'n_estiimators': 10, 'max_depth': 9, 'criterion': 'absolute_error'}. Best is trial 1 with value: 57.91427367693552.
[I 2025-04-04 14:33:49,495] Trial 3 finished with value: 58.384396585020184 and parameters: {'n_estiimators': 100, 'max_depth': 9, 'criterion': 'poisson'}. Best is trial 1 with value: 57.91427367693552.
[I 2025-04-04 14:33:52,024] Trial 4 finished with value: 57.96215

In [21]:
optuna.visualization.plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [n_estiimators (IntDistribution):
                                0.22682685549309087<extra></extra>, criterion
                                (CategoricalDistribution):
                                0.3670489462647241<extra></extra>, max_depth
                                (IntDistribution):
                                0.40612419824218504<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [0.23, 0.37, 0.41],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.22682685549309087, 0.3670489462647241, 0.40612419824218504],
              'y': [n_estiimators, criterion, max_depth]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [22]:
optuna.visualization.plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'y': [58.14466023072968, 57.91427367693552, 58.53303179991748,
                    58.384396585020184, 57.962151912719776, 58.89948637343897,
                    58.450225222294605, 58.971867304870884, 57.88414145416807,
                    58.128090557303665, 58.175407629700985, 57.2409642182432,
                    57.79107921453713, 58.00076529119476, 58.38213840730555,
                    57.88935849924807, 57.43012018817224, 58.318097525583525,
                    58.260514403590534, 58.37041882411448, 58.50054612898318,
                    58.88117329904395, 58.15953177486715, 58.09743198952701,
                    57.487080765857385, 58.15642139978195, 58.313362008354986,
                    58.009074835398295, 59.408254736957744, 57.74708854413166,
                    58.3997223170504, 57.897135654714575, 58.027377679238406,
                    58.197580229205506, 58.379623968024305, 57.83722467917725,
                    57.487080765857385, 57.40015423055185, 59.52805954056489,
                    58.87580962157519, 59.16762883994413, 58.01191846764635,
                    56.97433367498089, 58.156711201359975, 58.77621988078645,
                    57.47620505064437, 57.99915532916386, 58.17079432862329,
                    57.88431788151386, 57.47393391773934]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'y': [58.14466023072968, 57.91427367693552, 57.91427367693552,
                    57.91427367693552, 57.91427367693552, 57.91427367693552,
                    57.91427367693552, 57.91427367693552, 57.88414145416807,
                    57.88414145416807, 57.88414145416807, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    57.2409642182432, 57.2409642182432, 57.2409642182432,
                    56.97433367498089, 56.97433367498089, 56.97433367498089,
                    56.97433367498089, 56.97433367498089, 56.97433367498089,
                    56.97433367498089, 56.97433367498089]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [23]:
optuna.visualization.plot_parallel_coordinate(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'dimensions': [{'label': 'Objective Value',
                              'range': [56.97433367498089, 59.52805954056489],
                              'values': [58.14466023072968, 57.91427367693552,
                                         58.53303179991748, 58.384396585020184,
                                         57.962151912719776, 58.89948637343897,
                                         58.450225222294605, 58.971867304870884,
                                         57.88414145416807, 58.128090557303665,
                                         58.175407629700985, 57.2409642182432,
                                         57.79107921453713, 58.00076529119476,
                                         58.38213840730555, 57.88935849924807,
                                         57.43012018817224, 58.318097525583525,
                                         58.260514403590534, 58.37041882411448,
                                         58.50054612898318, 58.88117329904395,
                                         58.15953177486715, 58.09743198952701,
                                         57.487080765857385, 58.15642139978195,
                                         58.313362008354986, 58.009074835398295,
                                         59.408254736957744, 57.74708854413166,
                                         58.3997223170504, 57.897135654714575,
                                         58.027377679238406, 58.197580229205506,
                                         58.379623968024305, 57.83722467917725,
                                         57.487080765857385, 57.40015423055185,
                                         59.52805954056489, 58.87580962157519,
                                         59.16762883994413, 58.01191846764635,
                                         56.97433367498089, 58.156711201359975,
                                         58.77621988078645, 57.47620505064437,
                                         57.99915532916386, 58.17079432862329,
                                         57.88431788151386, 57.47393391773934]},
                             {'label': 'criterion',
                              'range': [0, 2],
                              'ticktext': [absolute_error, poisson, squared_error],
                              'tickvals': [0, 1, 2],
                              'values': [0, 1, 0, 1, 2, 1, 1, 2, 0, 2, 1, 0, 0, 1,
                                         0, 2, 2, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 0,
                                         1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 2, 1, 2, 1,
                                         2, 0, 1, 2, 1, 2, 0, 2]},
                             {'label': 'max_depth',
                              'range': [2, 10],
                              'values': [10, 3, 9, 9, 7, 5, 8, 9, 8, 6, 6, 6, 3,
                                         4, 8, 2, 5, 6, 6, 2, 7, 6, 10, 10, 5, 9,
                                         5, 3, 10, 3, 7, 3, 8, 5, 4, 7, 5, 4, 6,
                                         10, 8, 4, 3, 8, 9, 5, 4, 2, 9, 5]},
                             {'label': 'n_estiimators',
                              'range': [10, 100],
                              'values': [57, 73, 10, 100, 85, 23, 86, 17, 64, 32,
                                         92, 17, 94, 97, 28, 67, 83, 56, 79, 10,
                                         69, 21, 65, 17, 68, 46, 54, 12, 21, 37,
                                         77, 20, 51, 87, 14, 79, 68, 20, 12, 54,
                                         16, 41, 67, 46, 60, 90, 99, 91, 81, 91]}],
              'labelangle': 30,
              'labelside': 'bottom',
              'line': {'color': [58.14466023072968, 57.91427367693552,
                                 58.53303179991748, 58.384396585020184,
                                 57.962151912719776, 58.89948637343897,
                                 58.450225222294605, 58.971867304870884,
                

In [24]:
study.best_params

{'n_estiimators': 67, 'max_depth': 3, 'criterion': 'squared_error'}